In [1]:
#테스트 - 1 [드라이버 안에서 또 다른 드라이버 열고 닫고 가능한지 확인하는 테스트] 
#테스트 결과 : 성공
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import os
import pandas as pd

import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from PIL import Image
from io import BytesIO
import requests

print('='*100)
print('네이버 쇼핑 Best100 자료를 수집하는 크롤러')
print('='*100)

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
os.makedirs(f_dir+s+'-'+'네이버쇼핑Best100')
os.chdir(f_dir+s+'-'+'네이버쇼핑Best100')

f_name = s+'-'+'네이버쇼핑best100'+'.txt'
fc_name = s+'-'+'네이버쇼핑best100'+'.csv'
fx_name = s+'-'+'네이버쇼핑best100'+'.xlsx'


path = '/Users/kibeomkim/Desktop/chromedriver_88/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
#options.add_argument('headless')
driver = webdriver.Chrome(path, options=options)

s_time = time.time()

driver.get('https://search.shopping.naver.com/best100v2/main.nhn')
time.sleep(5)

html3 = driver.page_source
soup3 = BeautifulSoup(html3, 'html.parser')

category = []
number = 1
cat_list = soup3.find('ul', class_='category_area').find_all('img')
for c in cat_list : 
    cat_name = c['alt']
    category.append(cat_name)

def display() : 
    print(f'''
    {number:4}.{category[0]} {number+1:5}.{category[1]} {number+2:6}.{category[2]}
    {number+3:4}.{category[3]} {number+4:2}.{category[4]} {number+5:2}.{category[5]}
    {number+6:4}.{category[6]} {number+7:2}.{category[7]} {number+8:5}.{category[8]}
    ''')
display()

key_word = int(input('1.어떤 키워드로 조회하시겠습니까? 키워드 번호를 입력하세요(예 : 1) : '))
cnt = int(input('2.몇 건 조회하시겠습니까? '))
print()


driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div[1]/ul/li[%s]/a' %key_word).click()
time.sleep(5)




for b in range(0, 13) : 
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(3)


title1 = []
chep1 = []
people1 = []
grade1 = []
no1 = []
ch1 = []
img1 = []

no = 1
f = open(f_name, 'a', encoding='utf-8')



html0 = driver.page_source
soup0 = BeautifulSoup(html0, 'html.parser')
li_list = soup0.find('ul', 'type_normal').find_all('li', '_itemSection')

for a in li_list : 
    url = a.find('p', class_='cont').find('a')['href']
    path = '/Users/kibeomkim/Desktop/chromedriver_88/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')

    driver = webdriver.Chrome(path, options=options)
    
    driver.get(url)
    time.sleep(5)
    
    try : 
        driver.find_element_by_link_text('최저가순').click()
        time.sleep(5)
    except : 
        pass
    
    html1 = driver.page_source
    soup1 = BeautifulSoup(html1, 'html.parser')
    
    
    img_src = a.find('img', '_productLazyImg')['src']
    img1.append(img_src)
    
    print('-'*100)
    f.write('-'*100+'\n')
    
    print('1.판매순위 : %s' %no)
    f.write('1.판매순위 : %s' %no+'\n')
    no1.append(no)
    
    big_title = soup1.find('div', 'top_summary_title__15yAr').find('h2').get_text()
    title1.append(big_title)
    print('2.제품소개 : %s' %big_title)
    f.write('2.제품소개 : %s' %big_title+'\n')
    
    chep = soup1.find('div', class_='lowestPrice_low_price__fByaG').get_text().replace('\n', "").replace('\xa0', "").strip()
    chep1.append(chep)
    print('3.가격 : %s' %chep)
    f.write('3.가격 : %s' %chep+'\n')
    
    people = a.find('em').get_text().replace('(', "").replace(')', "").strip()
    people1.append(people)
    print('4.상품 평 수 : %s' %people)
    f.write('4.상품 평 수 : %s' %people+'\n')
    
    grade = soup1.find('div', 'top_grade__3jjdl').get_text().replace('평점', "").replace('\n', "").strip()
    grade2 = '5 / %s' %grade
    
    grade1.append(grade)
    print('5.평점 : %s' %grade2)
    f.write('5.평점 : %s' %grade2+'\n')
    

    try : 
        cheap = soup1.find('a', 'productByMall_mall__1ITj0').find('img')['alt']
        ch1.append(cheap)
        print('6.최저가 매장 명 : %s' %cheap)
        f.write('6.최저가 매장 명 : %s' %cheap+'\n')
    except : 
        cheap = soup1.find('a', 'productByMall_mall__1ITj0').get_text().replace('\n', "").strip()
        ch1.append(cheap)
        print('6.최저가 매장 명 : %s' %cheap)
        f.write('6.최저가 매장 명 : %s' %cheap+'\n')
    print('-'*100)
    f.write('-'*100+'\n')
    
    
    
    driver.quit()
    no +=1 
    if no > cnt : 
        break
f.close()
driver.quit()


naver = pd.DataFrame()
naver['판매순위'] = no1
naver['제품소개'] = title1
naver['최저가판매가격'] = chep1
naver['최저가판매매장'] = ch1
naver['상품평 갯수'] = people1
naver['상품평점(5점)'] = grade1

naver.to_csv(fc_name, encoding='utf-8-sig')
naver.to_excel(fx_name)


#엑셀에 사진 넣기 
wb = openpyxl.load_workbook(fx_name)
ws = wb.worksheets[0]

for a in range(2, no+1) : 
    ws.row_dimensions[a].height =  160
    
ws.column_dimensions['C'].width = 50


nor = 2
for a in img1 : 
    res = requests.get(a)
    image_data = BytesIO(res.content)
    img = openpyxl.drawing.image.Image(image_data)
    ws.add_image(img, anchor='C%s' %nor)
    nor +=1
wb.save(fx_name)
print('엑셀에 사진 첨부 완료') 


e_time = time.time()
total_time = e_time - s_time

print('='*100)
print('요청하신 자료 총 %s건을 수집했습니다' %cnt)
print('자료 수집에 소요된 시간은 %s초 입니다' %round(total_time, 2))
print('='*100)


네이버 쇼핑 Best100 자료를 수집하는 크롤러

       1.패션의류     2.패션잡화      3.화장품/미용
       4.디지털/가전  5.가구/인테리어  6.식품
       7.스포츠/레저  8.출산/육아     9.생활/건강
    
1.어떤 키워드로 조회하시겠습니까? 키워드 번호를 입력하세요(예 : 1) : 6
2.몇 건 조회하시겠습니까? 97

----------------------------------------------------------------------------------------------------
1.판매순위 : 1
2.제품소개 : 정관장 홍삼정 에브리타임 밸런스 10ml x 30포
3.가격 : 최저42,710원
4.상품 평 수 : 1
5.평점 : 5 / 4.8
6.최저가 매장 명 : YK STORE
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 2
2.제품소개 : CJ제일제당 비비고 사골곰탕 500g
3.가격 : 최저880원
4.상품 평 수 : 2
5.평점 : 5 / 4.9
6.최저가 매장 명 : 11번가
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 3
2.제품소개 : 지웨이 슈가 먹는 저분자 피쉬 콜라겐 펩타이드 150g
3.가격 : 최저14,900원
4.

----------------------------------------------------------------------------------------------------
1.판매순위 : 27
2.제품소개 : 맛있는 오뚜기밥 210g
3.가격 : 최저490원
4.상품 평 수 : 2
5.평점 : 5 / 4.8
6.최저가 매장 명 : 위메프
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 28
2.제품소개 : 종근당건강 프로메가 알티지 오메가3 520mg x 60캡슐
3.가격 : 최저12,400원
4.상품 평 수 : 2
5.평점 : 5 / 4.7
6.최저가 매장 명 : 더착한마트
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 29
2.제품소개 : 렛츠미 저분자 석류 콜라겐 젤리 20g x 50포
3.가격 : 최저25,900원
4.상품 평 수 : 2
5.평점 : 5 / 4.8
6.최저가 매장 명 : 더순수렛츠미
----------------------------------------------------------------------------------------------------
--------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
1.판매순위 : 54
2.제품소개 : ANCHOR 무염버터 454g
3.가격 : 최저3,900원
4.상품 평 수 : 5
5.평점 : 5 / 4.7
6.최저가 매장 명 : 맘쿠킹
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 55
2.제품소개 : 닥터리브 맥주효모 500mg x 60정
3.가격 : 최저9,800원
4.상품 평 수 : 5
5.평점 : 5 / 4.6
6.최저가 매장 명 : 스타유통
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 56
2.제품소개 : 정관장 홍삼원골드 50ml x 30포
3.가격 : 최저19,890원
4.상품 평 수 : 5
5.평점 : 5 / 4.8
6.최저가 매장 명 : 티몬
----------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
1.판매순위 : 81
2.제품소개 : 보르딘에프엔비 보르딘 콜드브루 더치커피 원액 25ml x 24개입 선물세트
3.가격 : 최저19,800원
4.상품 평 수 : 8
5.평점 : 5 / 4.9
6.최저가 매장 명 : 지니코지
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 82
2.제품소개 : NS WHEY PACK NS 포대유청 WPC 초코맛
3.가격 : 최저28,500원
4.상품 평 수 : 8
5.평점 : 5 / 4.8
6.최저가 매장 명 : 뉴트리션스토어
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위 : 83
2.제품소개 : CJ제일제당 비비고 한우 사골곰탕 500g
3.가격 : 최저1,400원
4.상품 평 수 : 8
5.평점 : 5 / 4.9
6.최저가 매장 명 : 코스트코구매대행 몽땅365
----------------------------------------------------------------------------------------------------
-----------------------------------